In [1]:
!pip install tensorflow_hub

In [2]:
import numpy as np
import time

import PIL.Image as Image
import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub

import datetime

%load_ext tensorboard

2024-02-27 03:54:13.327275: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-27 03:54:13.454644: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-27 03:54:13.985665: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-02-27 03:54:13.985712: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

In [3]:
AMAZON = "amazon"
DSLR = "dslr"
WEBCAM = "webcam"

DIR_RESULT = "result/cf/"

DIR_AMAZON = "office31/{}".format(AMAZON)
DIR_DSLR = "office31/{}".format(DSLR)
DIR_WEBCAM = "office31/{}".format(WEBCAM)

DIR_SOURCE = DIR_AMAZON
DIR_TARGET = DIR_WEBCAM

BATCH_SIZE = 64
IMAGE_SHAPE = (224, 224)

dataset_source =  tf.keras.utils.image_dataset_from_directory(
                                DIR_SOURCE,
                                validation_split=0.2,
                                subset="training",
                                seed=123,
                                image_size=IMAGE_SHAPE,
                                batch_size=BATCH_SIZE)

dataset_source_val =  tf.keras.utils.image_dataset_from_directory(
                                DIR_SOURCE,
                                validation_split=0.2,
                                subset="validation",
                                seed=123,
                                image_size=IMAGE_SHAPE,
                                batch_size=BATCH_SIZE)

dataset_target =  tf.keras.utils.image_dataset_from_directory(
                                DIR_TARGET,
                                validation_split=0.2,
                                subset="training",
                                seed=123,
                                image_size=IMAGE_SHAPE,
                                batch_size=BATCH_SIZE)

dataset_target_val =  tf.keras.utils.image_dataset_from_directory(
                                DIR_TARGET,
                                validation_split=0.2,
                                subset="validation",
                                seed=123,
                                image_size=IMAGE_SHAPE,
                                batch_size=BATCH_SIZE)

class_names = np.array(dataset_source.class_names)


Found 2817 files belonging to 31 classes.
Using 2254 files for training.


2024-02-27 03:54:15.157490: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-27 03:54:15.755211: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22822 MB memory:  -> device: 0, name: Quadro RTX 6000, pci bus id: 0000:37:00.0, compute capability: 7.5


Found 2817 files belonging to 31 classes.
Using 563 files for validation.
Found 795 files belonging to 31 classes.
Using 636 files for training.
Found 795 files belonging to 31 classes.
Using 159 files for validation.


In [4]:
normalization_layer = tf.keras.layers.Rescaling(1./255)

dataset_source = dataset_source.map(lambda x, y: (normalization_layer(x), y)) # Where x—images, y—labels.
dataset_source_val = dataset_source_val.map(lambda x, y: (normalization_layer(x), y)) # Where x—images, y—labels.

dataset_target = dataset_target.map(lambda x, y: (normalization_layer(x), y)) # Where x—images, y—labels.
dataset_target_val = dataset_target_val.map(lambda x, y: (normalization_layer(x), y)) # Where x—images, y—labels.


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [5]:
AUTOTUNE = tf.data.AUTOTUNE

dataset_source = dataset_source.cache().prefetch(buffer_size=AUTOTUNE)
dataset_source_val = dataset_source_val.cache().prefetch(buffer_size=AUTOTUNE)

dataset_target = dataset_target.cache().prefetch(buffer_size=AUTOTUNE)
dataset_target_val = dataset_target_val.cache().prefetch(buffer_size=AUTOTUNE)

In [28]:
mobilenet_v2 = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
inception_v3 = "https://tfhub.dev/google/tf2-preview/inception_v3/feature_vector/4"
vit_s16 = "https://www.kaggle.com/models/spsayakpaul/vision-transformer/frameworks/TensorFlow2/variations/vit-s16-fe/versions/1"

feature_extractor_model = vit_s16

In [29]:
feature_extractor_layer = hub.KerasLayer(
    feature_extractor_model,
    input_shape=(224, 224, 3),
    trainable=False)

In [30]:
num_classes = len(class_names)

model = tf.keras.Sequential([
  feature_extractor_layer,
  tf.keras.layers.Dense(num_classes)
])

model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_14 (KerasLayer)  (None, 384)              21665664  
                                                                 
 dense_14 (Dense)            (None, 31)                11935     
                                                                 
Total params: 21,677,599
Trainable params: 11,935
Non-trainable params: 21,665,664
_________________________________________________________________


In [31]:
model.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['acc'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir,
    histogram_freq=1) # Enable histogram computation for every epoch.

In [32]:
NUM_EPOCHS = 10

history = model.fit(dataset_source,
                    epochs=NUM_EPOCHS,
                    callbacks=tensorboard_callback)

Epoch 1/10


2024-02-27 04:19:58.437739: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:5340] Disabling cuDNN frontend for the following convolution:
  input: {count: 64 feature_map_count: 3 spatial: 224 224  value_min: 0.000000 value_max: 0.000000 layout: BatchDepthYX}
  filter: {output_feature_map_count: 384 input_feature_map_count: 3 layout: OutputInputYX shape: 16 16 }
  {zero_padding: 0 0  pad_alignment: default filter_strides: 16 16  dilation_rates: 1 1 }
  ... because it uses an identity activation.


 9/10 [==========================>...] - ETA: 0s - loss: 5.9564 - acc: 0.0833

2024-02-27 04:20:05.515811: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:5340] Disabling cuDNN frontend for the following convolution:
  input: {count: 60 feature_map_count: 3 spatial: 224 224  value_min: 0.000000 value_max: 0.000000 layout: BatchDepthYX}
  filter: {output_feature_map_count: 384 input_feature_map_count: 3 layout: OutputInputYX shape: 16 16 }
  {zero_padding: 0 0  pad_alignment: default filter_strides: 16 16  dilation_rates: 1 1 }
  ... because it uses an identity activation.


10/10 [==============================] - 18s 893ms/step - loss: 5.8047 - acc: 0.0881
Epoch 2/10
10/10 [==============================] - 3s 361ms/step - loss: 2.6501 - acc: 0.3286
Epoch 3/10
10/10 [==============================] - 4s 375ms/step - loss: 1.0624 - acc: 0.6934
Epoch 4/10
10/10 [==============================] - 3s 368ms/step - loss: 0.4433 - acc: 0.8884
Epoch 5/10
10/10 [==============================] - 3s 370ms/step - loss: 0.2190 - acc: 0.9513
Epoch 6/10
10/10 [==============================] - 3s 352ms/step - loss: 0.1287 - acc: 0.9764
Epoch 7/10
10/10 [==============================] - 3s 358ms/step - loss: 0.0855 - acc: 0.9921
Epoch 8/10
10/10 [==============================] - 4s 375ms/step - loss: 0.0621 - acc: 0.9953
Epoch 9/10
10/10 [==============================] - 3s 358ms/step - loss: 0.0482 - acc: 0.9969
Epoch 10/10
10/10 [==============================] - 3s 367ms/step - loss: 0.0391 - acc: 0.9984


In [33]:
source_acc = model.evaluate(dataset_source_val)
target_acc = model.evaluate(dataset_target_val)

2/3 [===================>..........] - ETA: 0s - loss: 0.1523 - acc: 0.9688

2024-02-27 04:20:47.727686: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:5340] Disabling cuDNN frontend for the following convolution:
  input: {count: 31 feature_map_count: 3 spatial: 224 224  value_min: 0.000000 value_max: 0.000000 layout: BatchDepthYX}
  filter: {output_feature_map_count: 384 input_feature_map_count: 3 layout: OutputInputYX shape: 16 16 }
  {zero_padding: 0 0  pad_alignment: default filter_strides: 16 16  dilation_rates: 1 1 }
  ... because it uses an identity activation.


1/2 [==============>...............] - ETA: 0s - loss: 0.3248 - acc: 0.9062

2024-02-27 04:20:51.869760: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:5340] Disabling cuDNN frontend for the following convolution:
  input: {count: 35 feature_map_count: 3 spatial: 224 224  value_min: 0.000000 value_max: 0.000000 layout: BatchDepthYX}
  filter: {output_feature_map_count: 384 input_feature_map_count: 3 layout: OutputInputYX shape: 16 16 }
  {zero_padding: 0 0  pad_alignment: default filter_strides: 16 16  dilation_rates: 1 1 }
  ... because it uses an identity activation.


2/2 [==============================] - 4s 4s/step - loss: 0.2612 - acc: 0.9293


In [34]:
history = model.fit(dataset_target,
                    epochs=NUM_EPOCHS,
                    callbacks=tensorboard_callback)

Epoch 1/10
6/7 [========================>.....] - ETA: 0s - loss: 0.1793 - acc: 0.9453

2024-02-27 04:20:57.355961: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:5340] Disabling cuDNN frontend for the following convolution:
  input: {count: 15 feature_map_count: 3 spatial: 224 224  value_min: 0.000000 value_max: 0.000000 layout: BatchDepthYX}
  filter: {output_feature_map_count: 384 input_feature_map_count: 3 layout: OutputInputYX shape: 16 16 }
  {zero_padding: 0 0  pad_alignment: default filter_strides: 16 16  dilation_rates: 1 1 }
  ... because it uses an identity activation.


7/7 [==============================] - 8s 1s/step - loss: 0.1809 - acc: 0.9449
Epoch 2/10
7/7 [==============================] - 3s 477ms/step - loss: 0.0833 - acc: 0.9799
Epoch 3/10
7/7 [==============================] - 3s 463ms/step - loss: 0.0417 - acc: 1.0000
Epoch 4/10
7/7 [==============================] - 3s 466ms/step - loss: 0.0281 - acc: 1.0000
Epoch 5/10
7/7 [==============================] - 3s 457ms/step - loss: 0.0218 - acc: 1.0000
Epoch 6/10
7/7 [==============================] - 3s 452ms/step - loss: 0.0176 - acc: 1.0000
Epoch 7/10
7/7 [==============================] - 3s 470ms/step - loss: 0.0146 - acc: 1.0000
Epoch 8/10
7/7 [==============================] - 3s 506ms/step - loss: 0.0126 - acc: 1.0000
Epoch 9/10
7/7 [==============================] - 3s 453ms/step - loss: 0.0112 - acc: 1.0000
Epoch 10/10
7/7 [==============================] - 3s 465ms/step - loss: 0.0102 - acc: 1.0000


In [35]:
source_acc_ft = model.evaluate(dataset_source_val)
target_acc_ft = model.evaluate(dataset_target_val)

2/2 [==============================] - 0s 71ms/step - loss: 0.0620 - acc: 0.9798


In [36]:
print(source_acc[1],target_acc[1], source_acc_ft[1], target_acc_ft[1])

0.955974817276001 0.9292929172515869 0.9685534834861755 0.9797979593276978


In [41]:
# !pip install tensorflow_hub
mobilenet_v2.split('/')[5]

'mobilenet_v2'

In [50]:
import numpy as np
import time

import PIL.Image as Image
import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub

import datetime


mobilenet_v2 = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
inception_v3 = "https://tfhub.dev/google/tf2-preview/inception_v3/feature_vector/4"
vit_s16 = "https://www.kaggle.com/models/spsayakpaul/vision-transformer/frameworks/TensorFlow2/variations/vit-s16-fe/versions/1"
efficiennet_v2 = "https://www.kaggle.com/models/google/efficientnet-v2/frameworks/TensorFlow2/variations/imagenet1k-b0-feature-vector/versions/2"
resnet_50 = "https://www.kaggle.com/models/tensorflow/resnet-50/frameworks/TensorFlow2/variations/feature-vector/versions/1"

fe_model = [mobilenet_v2, inception_v3, vit_s16, efficiennet_v2, resnet_50]

AMAZON = "amazon"
DSLR = "dslr"
WEBCAM = "webcam"

DIR_RESULT = "result/cf/"

DIR_AMAZON = "office31/{}".format(AMAZON)
DIR_DSLR = "office31/{}".format(DSLR)
DIR_WEBCAM = "office31/{}".format(WEBCAM)


BATCH_SIZE = 64
IMAGE_SHAPE = (224, 224)

model_ = []
source = []
target = []
source_acc_ = []
target_acc_ = []
training_time_ = []
source_acc_ft_ = []
target_acc_ft_ = []
finetuning_time_ = []
####################################################

# feature_extractor_model = mobilenet_v2

for feature_extractor_model in fe_model:
    
    
    for DIR_SOURCE in [DIR_AMAZON, DIR_DSLR, DIR_WEBCAM]:
        for DIR_TARGET in [DIR_AMAZON, DIR_DSLR, DIR_WEBCAM]:

            if DIR_SOURCE == DIR_TARGET:
                continue
                
            model_ += [feature_extractor_model.split('/')[5]]
            source += [DIR_SOURCE.split('/')[1]] 
            target += [DIR_TARGET.split('/')[1]]

            print(source[-1], target[-1])

    # DIR_SOURCE = DIR_AMAZON
    # DIR_TARGET = DIR_WEBCAM

    ##################################################


            dataset_source =  tf.keras.utils.image_dataset_from_directory(
                                            DIR_SOURCE,
                                            validation_split=0.2,
                                            subset="training",
                                            seed=123,
                                            image_size=IMAGE_SHAPE,
                                            batch_size=BATCH_SIZE)

            dataset_source_val =  tf.keras.utils.image_dataset_from_directory(
                                            DIR_SOURCE,
                                            validation_split=0.2,
                                            subset="validation",
                                            seed=123,
                                            image_size=IMAGE_SHAPE,
                                            batch_size=BATCH_SIZE)

            dataset_target =  tf.keras.utils.image_dataset_from_directory(
                                            DIR_TARGET,
                                            validation_split=0.2,
                                            subset="training",
                                            seed=123,
                                            image_size=IMAGE_SHAPE,
                                            batch_size=BATCH_SIZE)

            dataset_target_val =  tf.keras.utils.image_dataset_from_directory(
                                            DIR_TARGET,
                                            validation_split=0.2,
                                            subset="validation",
                                            seed=123,
                                            image_size=IMAGE_SHAPE,
                                            batch_size=BATCH_SIZE)

            class_names = np.array(dataset_source.class_names)

            normalization_layer = tf.keras.layers.Rescaling(1./255)

            dataset_source = dataset_source.map(lambda x, y: (normalization_layer(x), y)) # Where x—images, y—labels.
            dataset_source_val = dataset_source_val.map(lambda x, y: (normalization_layer(x), y)) # Where x—images, y—labels.

            dataset_target = dataset_target.map(lambda x, y: (normalization_layer(x), y)) # Where x—images, y—labels.
            dataset_target_val = dataset_target_val.map(lambda x, y: (normalization_layer(x), y)) # Where x—images, y—labels.

            AUTOTUNE = tf.data.AUTOTUNE

            dataset_source = dataset_source.cache().prefetch(buffer_size=AUTOTUNE)
            dataset_source_val = dataset_source_val.cache().prefetch(buffer_size=AUTOTUNE)

            dataset_target = dataset_target.cache().prefetch(buffer_size=AUTOTUNE)
            dataset_target_val = dataset_target_val.cache().prefetch(buffer_size=AUTOTUNE)

            feature_extractor_layer = hub.KerasLayer(
                feature_extractor_model,
                input_shape=(224, 224, 3),
                trainable=False)

            num_classes = len(class_names)

            model = tf.keras.Sequential([
              feature_extractor_layer,
              tf.keras.layers.Dense(num_classes)
            ])

            model.compile(
              optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['acc'])

            NUM_EPOCHS = 10

            # training 
            ###############################################################
            start_time = time.time()
            history = model.fit(dataset_source,
                                epochs=NUM_EPOCHS,
                               verbose=0)
            end_time = time.time()
            elapsed_time = end_time - start_time
            training_time_ += [elapsed_time]

            source_acc = model.evaluate(dataset_source_val, verbose=0)
            target_acc = model.evaluate(dataset_target_val, verbose=0)
            ###############################################################

            #fine tuning
            ###############################################################
            start_time = time.time()
            history = model.fit(dataset_target,
                                epochs=NUM_EPOCHS,
                                verbose=0)
            end_time = time.time()
            elapsed_time = end_time - start_time
            finetuning_time_ += [elapsed_time]

            source_acc_ft = model.evaluate(dataset_source_val, verbose=0)
            target_acc_ft = model.evaluate(dataset_target_val, verbose=0)
            ###############################################################

            source_acc_ += [source_acc[1]]
            target_acc_ += [target_acc[1]]
            source_acc_ft_ += [source_acc_ft[1]]
            target_acc_ft_ += [target_acc_ft[1]]

            print(source_acc[1],target_acc[1], source_acc_ft[1], target_acc_ft[1])

amazon dslr
Found 2817 files belonging to 31 classes.
Using 2254 files for training.
Found 2817 files belonging to 31 classes.
Using 563 files for validation.
Found 498 files belonging to 31 classes.
Using 399 files for training.
Found 498 files belonging to 31 classes.
Using 99 files for validation.
0.8579040765762329 0.7878788113594055 0.8223801255226135 0.9797979593276978
amazon webcam
Found 2817 files belonging to 31 classes.
Using 2254 files for training.
Found 2817 files belonging to 31 classes.
Using 563 files for validation.
Found 795 files belonging to 31 classes.
Using 636 files for training.
Found 795 files belonging to 31 classes.
Using 159 files for validation.
0.8614564538002014 0.7169811129570007 0.8223801255226135 0.9685534834861755
dslr amazon
Found 498 files belonging to 31 classes.
Using 399 files for training.
Found 498 files belonging to 31 classes.
Using 99 files for validation.
Found 2817 files belonging to 31 classes.
Using 2254 files for training.
Found 2817 fi

2024-02-27 04:57:34.769983: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:5340] Disabling cuDNN frontend for the following convolution:
  input: {count: 14 feature_map_count: 3 spatial: 224 224  value_min: 0.000000 value_max: 0.000000 layout: BatchDepthYX}
  filter: {output_feature_map_count: 384 input_feature_map_count: 3 layout: OutputInputYX shape: 16 16 }
  {zero_padding: 0 0  pad_alignment: default filter_strides: 16 16  dilation_rates: 1 1 }
  ... because it uses an identity activation.
2024-02-27 04:58:23.088011: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:5340] Disabling cuDNN frontend for the following convolution:
  input: {count: 51 feature_map_count: 3 spatial: 224 224  value_min: 0.000000 value_max: 0.000000 layout: BatchDepthYX}
  filter: {output_feature_map_count: 384 input_feature_map_count: 3 layout: OutputInputYX shape: 16 16 }
  {zero_padding: 0 0  pad_alignment: default filter_strides: 16 16  dilation_rates: 1 1 }
  ... because it uses an

0.8596802949905396 0.7979797720909119 0.8401420712471008 0.9595959782600403
amazon webcam
Found 2817 files belonging to 31 classes.
Using 2254 files for training.
Found 2817 files belonging to 31 classes.
Using 563 files for validation.
Found 795 files belonging to 31 classes.
Using 636 files for training.
Found 795 files belonging to 31 classes.
Using 159 files for validation.
0.8650088906288147 0.6918238997459412 0.8401420712471008 0.9874213933944702
dslr amazon
Found 498 files belonging to 31 classes.
Using 399 files for training.
Found 498 files belonging to 31 classes.
Using 99 files for validation.
Found 2817 files belonging to 31 classes.
Using 2254 files for training.
Found 2817 files belonging to 31 classes.
Using 563 files for validation.
0.8787878751754761 0.5435168743133545 0.9494949579238892 0.8632326722145081
dslr webcam
Found 498 files belonging to 31 classes.
Using 399 files for training.
Found 498 files belonging to 31 classes.
Using 99 files for validation.
Found 795 

In [47]:
# model_ = ['mobilenet_v2']*6 + ['inception_v3']*6

In [51]:
import pandas as pd

d = {'model': model_,
     'source': source, 
     'target': target, 
     'source_acc': source_acc_,
     'target_acc': target_acc_,
     'source_acc_ft':source_acc_ft_,
     'target_acc_ft': target_acc_ft_,
     'training_time': training_time_,
     'finetuning_time': finetuning_time_}

df = pd.DataFrame(data=d)

In [55]:
df['catastrophic forgetting'] = df['source_acc_ft'] - df['source_acc']

In [56]:
df

,model,source,target,source_acc,target_acc,source_acc_ft,target_acc_ft,training_time,finetuning_time,catastrophic forgetting
0,mobilenet_v2,amazon,dslr,0.857904,0.787879,0.822380,0.979798,20.085061,3.320356,-0.035524
1,mobilenet_v2,amazon,webcam,0.861456,0.716981,0.822380,0.968553,18.317163,4.713650,-0.039076
2,mobilenet_v2,dslr,amazon,0.979798,0.591474,0.939394,0.861456,5.416722,15.983193,-0.040404
3,mobilenet_v2,dslr,webcam,0.939394,0.930818,0.979798,0.981132,5.632095,4.717780,0.040404
4,mobilenet_v2,webcam,amazon,0.968553,0.607460,0.911950,0.857904,6.923398,15.200419,-0.056604
5,mobilenet_v2,webcam,dslr,0.987421,1.000000,0.993711,0.989899,6.703024,3.112073,0.006289
6,inception_v3,amazon,dslr,0.840142,0.787879,0.834813,0.949495,25.773040,4.370522,-0.005329
7,inception_v3,amazon,webcam,0.843695,0.773585,0.831261,0.968553,26.439016,6.580364,-0.012433
8,inception_v3,dslr,amazon,0.949495,0.644760,0.888889,0.838366,7.696542,22.495136,-0.060606
9,inception_v3,dslr,webcam,0.959596,0.930818,0.979798,0.968553,7.795503,6.478355,0.020202


In [61]:
df.drop(columns=['source','target']).groupby('model').mean()

,source_acc,target_acc,source_acc_ft,target_acc_ft,training_time,finetuning_time,catastrophic forgetting
model,,,,,,,
efficientnet-v2,0.920877,0.783307,0.903185,0.920538,14.709051,9.348823,-0.017692
inception_v3,0.924817,0.790048,0.903592,0.925039,14.563201,11.107593,-0.021225
mobilenet_v2,0.932421,0.772435,0.911602,0.939791,10.512910,7.841245,-0.020819
resnet-50,0.933765,0.752134,0.924220,0.943237,19.299733,15.595271,-0.009546
vision-transformer,0.906405,0.727519,0.920890,0.939544,30.203264,24.624377,0.014485


In [63]:
df.drop(columns=['model']).groupby(['source','target']).mean()

source_acc  target_acc  source_acc_ft  target_acc_ft  \
source target                                                         
amazon dslr      0.852931    0.779798       0.835879       0.951515   
       webcam    0.854707    0.723270       0.831616       0.976101   
dslr   amazon    0.943434    0.600710       0.919192       0.857194   
       webcam    0.947475    0.911950       0.983838       0.977359   
webcam amazon    0.973585    0.601066       0.918239       0.855773   
       dslr      0.969811    0.973737       0.987421       0.983838   

               training_time  finetuning_time  catastrophic forgetting  
source target                                                           
amazon dslr        31.927634         5.764215                -0.017052  
       webcam      31.503492         8.404114                -0.023091  
dslr   amazon       9.355187        27.071064                -0.024242  
       webcam       9.810508         8.214898                 0.036364  
webcam amazon      11.970888        27.166291                -0.055346  
       dslr        12.578082         5.600188                 0.017610

In [69]:
df.drop(df[(df['source'] == 'dslr') & (df['target'] == 'webcam')].index).drop(columns=['source','target']).groupby('model').mean()

,source_acc,target_acc,source_acc_ft,target_acc_ft,training_time,finetuning_time,catastrophic forgetting
model,,,,,,,
efficientnet-v2,0.915153,0.752547,0.887862,0.909677,15.723103,10.112403,-0.027292
inception_v3,0.917861,0.761894,0.888351,0.916337,15.916741,12.033440,-0.029510
mobilenet_v2,0.931027,0.740759,0.897963,0.931522,11.489073,8.465938,-0.033064
resnet-50,0.928599,0.711366,0.911084,0.933143,21.208681,16.907024,-0.017515
vision-transformer,0.901827,0.712016,0.907088,0.933742,32.997684,26.487067,0.005260


In [70]:
df

,model,source,target,source_acc,target_acc,source_acc_ft,target_acc_ft,training_time,finetuning_time,catastrophic forgetting
0,mobilenet_v2,amazon,dslr,0.857904,0.787879,0.822380,0.979798,20.085061,3.320356,-0.035524
1,mobilenet_v2,amazon,webcam,0.861456,0.716981,0.822380,0.968553,18.317163,4.713650,-0.039076
2,mobilenet_v2,dslr,amazon,0.979798,0.591474,0.939394,0.861456,5.416722,15.983193,-0.040404
3,mobilenet_v2,dslr,webcam,0.939394,0.930818,0.979798,0.981132,5.632095,4.717780,0.040404
4,mobilenet_v2,webcam,amazon,0.968553,0.607460,0.911950,0.857904,6.923398,15.200419,-0.056604
5,mobilenet_v2,webcam,dslr,0.987421,1.000000,0.993711,0.989899,6.703024,3.112073,0.006289
6,inception_v3,amazon,dslr,0.840142,0.787879,0.834813,0.949495,25.773040,4.370522,-0.005329
7,inception_v3,amazon,webcam,0.843695,0.773585,0.831261,0.968553,26.439016,6.580364,-0.012433
8,inception_v3,dslr,amazon,0.949495,0.644760,0.888889,0.838366,7.696542,22.495136,-0.060606
9,inception_v3,dslr,webcam,0.959596,0.930818,0.979798,0.968553,7.795503,6.478355,0.020202


In [71]:
df.to_csv('catastrophic_forgetting_office31.csv')